In [1]:
# automatically reload dependant notebooks
%load_ext autoreload
%autoreload 2
import import_ipynb

# Maximum Flow

Given a flow network $G = (V, E)$ with source vertex $s$ and target vertex $t$, Maximum Flow (MF) algorithm iteratively augment the flow so as to maximise the total flow from $s$ to $t$. See CLRS 4ed Chapter 24 *Maximum Flow* p.670. MF could be used in logistics to maximise the throughput from a factory to a warehouse through a transportation network.

## flow network

MF algorithms use directed edges with a flow and the maximum capacity. So, we define the weighted edge `FlowEdge` and `FlowGraph` which uses this new edge.

In [2]:
from graph import *
from util import *

class FlowEdge(Edge):
  def __init__(self, u: Vert, v: Vert, cap: float = 0.0):
    super().__init__(u, v)
    self.flo: float = 0.0
    self.cap: float = cap

  def residual(self) -> float: return self.cap - self.flo

  def __str__(self) -> str: return f"{self.tag}: {self.showFlow()}"
  def show(self) -> str: return f"{self.showFlow()}"
  def showFlow(self) -> str: return f"{self.flo}{f'/{self.cap}' if self.cap > 0.0 else ''}"

class FlowGraph(LstGraph):  # uses FlowEdge
  def __init__(self, tag: Tag):
    super().__init__(tag)

  def makeVEc(self, vt: [Tag], et: [Tag], ec: {Tag, float}) -> None:
    self.makeV(vt)
    self.makeEc(et, ec)
  def makeEc(self, et: [Tag], ec: {Tag, float}) -> None:
    for etag in et:
      [utag, vtag] = parseETag(etag)
      e = FlowEdge(self.getV(utag), self.getV(vtag), float(ec[etag]))
      self.ee[e.tag] = e

importing Jupyter notebook from graph.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
importing Jupyter notebook from util.ipynb


## Edmonds-Karp algorithm

The general idea of MF algorithm, which CLRS calls "Ford-Fulkerson method", is iteratively to increase the total flow $f$ from source $s$ to target $t$, with each iteration increasing $f$ in the flow network $G$ by finding an augmenting path $p$ in an associated residual network $G_f$. See p.685-689.

In a flow network $G$ with a flow $f$, a residual network $G_f$ consists of edges whose capacities collectively indicate a way in which $f$ can be increased in $G$. See p.677. An augmenting path $p$ is a simple path from source $s$ to target $t$ in the residual network $G_f$. The capacity of the edge with the minimum capacity amongst all the edges along $p$ is called the residual capacity of $p$. Therefore, the residual capacity of a path is its choke point.

CLRS presents a basic Ford-Fulkerson maximum flow algorithm on p.685. But this algorithm's runtime $O(Ef^*)$ depends on the maximum flow $f^*$. See p.688. With a minor tweak, namely applying a BFS to the residual network to discover an augmenting path, the flow-independent runtime $O(VE^2)$ is obtained. This tweaked version of Ford-Fulkerson is called Edmonds-Karp. Below, we implement Edmonds-Karp. See p.689.

In [3]:
from ega import *

def mfEdmondsKarp(fn: FlowGraph, s: Vert, t: Vert) -> FlowGraph:
  def resNet(fn: FlowGraph) -> FlowGraph:
    # extract a residual network from flow network fn; see p.677
    def edgeResCap(fn: FlowGraph, u: Vert, v: Vert) -> float:
      # compute residual capacity for edge e; see Equation 24.2 p.677
      uvtag = makeETag(u, v)
      vutag = makeETag(v, u)
      if fn.hasE(uvtag): return fn.getE(uvtag).residual()
      elif fn.hasE(vutag): return fn.getE(vutag).flo
      else: return 0.0

    rn = FlowGraph(f"{fn.tag}-")
    rn.dupVV(fn.vv)
    for u in fn.getVV():
      for v in fn.getVV():
        if fn.hasE(makeETag(u, v)) and (r := edgeResCap(fn, u, v)) > 0.0: rn.insE(FlowEdge(u, v, r))
    return rn

  def augPath(rn: FlowGraph, s: Vert, t: Vert) -> [FlowEdge]:
    # find an augmenting path from vertex s to vertex t in residual network rn; see p.681
    vv = list(reversed(rn.pathSV(s, t)))
    return list(map(lambda uv: rn.getE(makeETag(uv[0], uv[1])), zip(vv, vv[1:])))

  def pathResCap(ap: [FlowEdge]) -> float:
    # residual capacity (the minimum) of augmenting path ap; see p.681
    if len(ap) == 1: return 0.0;
    return min([e.residual() for e in ap])

  # initialize
  for e in fn.getEE(): e.flo = 0.0
  # augment flow
  while ap := augPath(bfs(resNet(fn), s), s, t):
    c = pathResCap(ap)
    for e in ap:
      if fn.hasE(e.tag):
        uv = fn.getE(e.tag)
        uv.flo += c
      else:
        vu = fn.getE(makeETag(e.v, e.u))
        vu.flo -= c
  fn.tag = f"{fn.tag}➨"
  return fn

importing Jupyter notebook from ega.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Conclusion

In this notebook, we implemented Edmonds-Karp maximum flow algorithm and the maximum bipartite matching algorithm, which uses the maximum flow algorithm. Tests for these algorithms are in [`flowtest.ipynb`](./flowtest.ipynb).